In [ ]:
'''
this script combine the movies both in movielens and having scripts,
i) the sentiment arc of every selected movie is found in '../data/springField_scraping/texts/'
ii) the genome is found in '../movielens/ml-latest/genome-scores.csv'
iii) select movies table is found in ../movielens/ml-latest/movies.csv'
------
the progression to combine:
1) select genomes of movies in which the sentiment scripts exist and 
saved as '../movielens/ml-latest/select-genome-scores.csv'.
2) used One-hot encoding to transder the genome(codes as words to bits(0 and 1))and
then saved to ('../movielens/ml-latest/relevance_df.csv').
3) find out the ids of selected movies with genome,and created a dictionaary for movie id to genome index and dumped to
('../ids2genome_sim_index.pkl');
4) find the selected movies having genome data and emotional arc,counting 3767.
5)selected movies having emotional arc and genome data,then saved to  '../data/normed_sentiment_with_genome/'+val)
------
'''

In [13]:
import pandas as pd
import numpy as np
import os
import string
from shutil import copyfile
from collections import defaultdict
import pickle

In [2]:
movies_with_sentiment=[i[:-4] for i in os.listdir('../data/normed_sentiment/')]
genome=pd.read_csv('../movielens/ml-latest/genome-scores.csv')
select_movies=pd.read_csv('../movielens/ml-latest/select-movies.csv')


In [3]:
final_movies=select_movies[select_movies['title_without_year'].isin(movies_with_sentiment)]
final_movie_ids=final_movies['movieId'].tolist()
final_select_genome=genome[genome['movieId'].isin(final_movie_ids)]
print("final selected movie's genome numbers is:{}".format(final_select_genome.shape[0]))

final selected movie's genome numbers is:4463496


In [4]:
final_select_genome.to_csv('../movielens/ml-latest/select-genome-scores.csv',index=False)

In [5]:
relevance=defaultdict(list)
for i in final_select_genome['movieId'].unique().tolist():
    relevance[i]=[0]*1128
    one_movie_genome=final_select_genome[final_select_genome['movieId']==i]
    for _,row in one_movie_genome.iterrows():
        relevance[i][int(row['tagId'])-1]=row['relevance']
    
relevance_df=pd.DataFrame.from_dict(relevance)
relevance_df=relevance_df.T 
relevance_df.to_csv('../movielens/ml-latest/relevance_df.csv')

In [8]:
genome_ids=final_select_genome['movieId'].unique().tolist()

In [10]:
ids2genome_sim_index={}
for index,ids in enumerate(genome_ids):
    ids2genome_sim_index[ids]=index
    

In [14]:
with open('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/ids2genome_sim_index.pkl','wb') as f:
        pickle.dump(ids2genome_sim_index, f, pickle.HIGHEST_PROTOCOL)

In [33]:
#with open('/Users/yangyangwang/PycharmProjects/movie_project/app/static/mood/title2id.pkl','rb') as f:
 #   t=pickle.load(f)


In [52]:
relevance_df.shape

(3957, 1128)

In [50]:
movie_genome_sentiment=final_movies[final_movies['movieId'].isin(genome_ids)].reset_index().drop('index',axis=1)

In [63]:
select_movies_with_sentiment=set(movies_with_sentiment).intersection(set(movie_genome_sentiment['title_without_year'].tolist()))

In [64]:
print("the number of selected movies with emotional arc  is:{}".format(len(select_movies_with_sentiment))

3767

In [67]:
for one in select_movies_with_sentiment:
    val=one+".txt"
    
    try:
        copyfile('../data/normed_sentiment/'+val,'../data/normed_sentiment_with_genome/'+val)
    except IOError: 
        pass

In [71]:
len(os.listdir('../data/normed_sentiment_with_genome'))

3767